In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
from datetime import datetime
import logging
from pathlib import Path 
import concurrent.futures
import tqdm
import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime
from pathlib import Path 

In [2]:
# Read the csv from stagging
path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)
ticker = 'NHTC'

In [3]:
ticker_file = ticker_file_all[ticker_file_all['ticker'] == ticker]

In [4]:
from yahoofinancials import YahooFinancials
from datetime import datetime
import pandas as pd


# Get today's date
end_date = datetime.today().strftime('%Y-%m-%d')

# Set a very early start date
start_date = '1900-01-01'

# Create YahooFinancials object
yahoo_financials = YahooFinancials(ticker)

# Retrieve monthly historical prices
data = yahoo_financials.get_historical_price_data(start_date, end_date, 'monthly')

# Extract the historical prices
historical_prices = data[ticker]['prices']

# Create a DataFrame from the historical prices
hist = pd.DataFrame(historical_prices)

# Capitalize first letter of each column
hist.columns = hist.columns.str.title()

hist.drop(columns='Date', inplace=True)
# Rename the "Formatted_Date" column to "Date"
hist = hist.rename(columns={'Formatted_Date': 'Date'})

# Print the updated DataFrame
hist['Date'] = pd.to_datetime(hist['Date'])
hist['year'] = hist['Date'].dt.year
hist.dropna(inplace=True)

hist



,High,Low,Open,Close,Volume,Adjclose,Date,year
0,14500.000,12000.00,13000.00,12750.00,792,6064.383789,1995-07-01,1995
1,17000.000,12750.00,14500.00,14500.00,574,6896.749023,1995-08-01,1995
2,17500.000,14000.00,14500.00,15250.00,548,7253.479492,1995-09-01,1995
3,17000.000,14500.00,15250.00,16000.00,523,7610.206543,1995-10-01,1995
4,28500.000,15500.00,17000.00,26000.00,864,12366.584961,1995-11-01,1995
...,...,...,...,...,...,...,...,...
332,5.750,4.50,5.10,4.94,342900,4.765749,2023-03-01,2023
333,6.250,4.94,4.94,6.17,196400,5.952363,2023-04-01,2023
334,6.200,4.61,6.16,5.50,383500,5.305996,2023-05-01,2023
335,5.630,4.90,5.51,5.15,229400,5.150000,2023-06-01,2023


In [5]:
# Convert 'Date' column in 'hist' dataframe to string format
hist['Date'] = hist['Date'].astype(str)
# Extract the 'YYYY-MM' format from 'Date' column in 'ticker_file' dataframe
ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
# Filter 'hist' dataframe based on 'YYYY-MM' values from 'ticker_file'
filtered_hist = hist[hist['Date'].str.slice(0, 7).isin(ticker_file['YearMonth'])]

C:\Users\germa\AppData\Local\Temp/ipykernel_16368/3425731416.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)


In [6]:
ticker_file['YearMonth']

34029    2010-12
34030    2011-12
34031    2012-12
34032    2013-12
34033    2014-12
34034    2015-12
34035    2016-12
34036    2017-12
34037    2018-12
34038    2019-12
34039    2020-12
34040    2021-12
34041    2022-12
Name: YearMonth, dtype: object

In [7]:
filtered_hist

,High,Low,Open,Close,Volume,Adjclose,Date,year
185,0.150000,0.090000,0.090000,0.120000,148500,0.057077,2010-12-01,2010
197,0.990000,0.600000,0.950000,0.650000,204000,0.309165,2011-12-01,2011
209,1.250000,0.720000,0.850000,1.010000,150100,0.480394,2012-12-01,2012
221,3.480000,3.020000,3.390000,3.200000,228900,1.522041,2013-12-01,2013
233,13.950000,11.010000,13.600000,11.450000,545500,5.464943,2014-12-01,2014
245,50.000000,33.509998,48.590000,33.529999,7341200,16.075354,2015-12-01,2015
257,27.500000,24.000000,25.209999,24.850000,1729000,12.199968,2016-12-01,2016
269,18.000000,15.070000,18.000000,15.190000,2107600,7.950179,2017-12-01,2017
281,23.290001,18.410000,21.500000,18.490000,1642000,11.035346,2018-12-01,2018
293,6.080000,5.060000,5.810000,5.380000,784900,3.461697,2019-12-01,2019


In [8]:
# Format to datetime
filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])
hist['Date'] = pd.to_datetime(hist['Date'])
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year > earliest_date.year]
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year < earliest_date.year]

C:\Users\germa\AppData\Local\Temp/ipykernel_16368/2001008434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])


In [9]:
filtered_hist_after_earliest

,High,Low,Open,Close,Volume,Adjclose,Date,year
0,14500.00,12000.00,13000.00,12750.00,792,6064.383789,1995-07-01,1995
1,17000.00,12750.00,14500.00,14500.00,574,6896.749023,1995-08-01,1995
2,17500.00,14000.00,14500.00,15250.00,548,7253.479492,1995-09-01,1995
3,17000.00,14500.00,15250.00,16000.00,523,7610.206543,1995-10-01,1995
4,28500.00,15500.00,17000.00,26000.00,864,12366.584961,1995-11-01,1995
...,...,...,...,...,...,...,...,...
169,1.06,0.55,0.66,0.70,1055500,0.332947,2009-08-01,2009
170,0.74,0.48,0.70,0.50,451400,0.237819,2009-09-01,2009
171,0.79,0.43,0.50,0.46,510900,0.218793,2009-10-01,2009
172,0.60,0.15,0.48,0.24,1094700,0.114153,2009-11-01,2009


In [10]:
# Filter the resulting dataframe for rows with the same month as the earliest date
appended_hist = filtered_hist_after_earliest[filtered_hist_after_earliest['Date'].dt.month == earliest_date.month]

In [11]:
appended_hist

,High,Low,Open,Close,Volume,Adjclose,Date,year
5,28000.000,21500.0000,26000.0000,21500.0000,760,10226.214844,1995-12-01,1995
17,12000.000,4000.0000,11000.0000,5500.0000,329,2616.008545,1996-12-01,1996
29,375.000,125.0000,250.0000,187.5000,1749,89.182114,1997-12-01,1997
41,412.500,318.7500,318.7500,387.5000,21139,184.309708,1998-12-01,1998
53,231.250,131.2500,231.2500,175.0000,14328,83.236618,1999-12-01,1999
65,3.125,1.5625,1.5625,1.5625,21806,0.743184,2000-12-01,2000
77,4.000,2.0000,4.0000,3.0000,256538,1.426914,2001-12-01,2001
89,3.000,1.0000,3.0000,2.0000,636654,0.951276,2002-12-01,2002
101,11.100,7.9100,8.1000,10.5000,968400,4.994196,2003-12-01,2003
113,12.280,8.7000,10.3000,11.0000,729300,5.232018,2004-12-01,2004


In [12]:
# Concatenate the filtered_hist dataframe with the appended_hist dataframe
prices_filtered = pd.concat([filtered_hist, appended_hist]).sort_values(by='year').reset_index(drop=True)

In [13]:
prices_filtered

,High,Low,Open,Close,Volume,Adjclose,Date,year
0,28000.000000,21500.000000,26000.000000,21500.000000,760,10226.214844,1995-12-01,1995
1,12000.000000,4000.000000,11000.000000,5500.000000,329,2616.008545,1996-12-01,1996
2,375.000000,125.000000,250.000000,187.500000,1749,89.182114,1997-12-01,1997
3,412.500000,318.750000,318.750000,387.500000,21139,184.309708,1998-12-01,1998
4,231.250000,131.250000,231.250000,175.000000,14328,83.236618,1999-12-01,1999
5,3.125000,1.562500,1.562500,1.562500,21806,0.743184,2000-12-01,2000
6,4.000000,2.000000,4.000000,3.000000,256538,1.426914,2001-12-01,2001
7,3.000000,1.000000,3.000000,2.000000,636654,0.951276,2002-12-01,2002
8,11.100000,7.910000,8.100000,10.500000,968400,4.994196,2003-12-01,2003
9,12.280000,8.700000,10.300000,11.000000,729300,5.232018,2004-12-01,2004


In [14]:
prices_filtered['ticker'] = ticker
prices_filtered['Growth -1'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=1)) /  prices_filtered['Close'].shift(periods=1)
prices_filtered['Growth +1'] = (prices_filtered['Close'].shift(periods=-1) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['Growth +5'] = (prices_filtered['Close'].shift(periods=-5) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['Growth -10'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=5)) /  prices_filtered['Close'].shift(periods=5)
prices_filtered['Growth -5'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=10)) /  prices_filtered['Close'].shift(periods=10)

# filter_years = pd.to_datetime(ticker_file['Date']).dt.year.tolist()
# histgrouped_filtered = prices_filtered[prices_filtered['year'].isin(filter_years)]
prices_filtered.reset_index(names=['longevity'], inplace=True)

In [15]:
prices_filtered.head(15)

,longevity,High,Low,Open,Close,Volume,Adjclose,Date,year,ticker,Growth -1,Growth +1,Growth +5,Growth -10,Growth -5
0,0,28000.000,21500.0000,26000.0000,21500.0000,760,10226.214844,1995-12-01,1995,NHTC,NaN,-0.744186,-0.999927,NaN,NaN
1,1,12000.000,4000.0000,11000.0000,5500.0000,329,2616.008545,1996-12-01,1996,NHTC,-0.744186,-0.965909,-0.999455,NaN,NaN
2,2,375.000,125.0000,250.0000,187.5000,1749,89.182114,1997-12-01,1997,NHTC,-0.965909,1.066667,-0.989333,NaN,NaN
3,3,412.500,318.7500,318.7500,387.5000,21139,184.309708,1998-12-01,1998,NHTC,1.066667,-0.548387,-0.972903,NaN,NaN
4,4,231.250,131.2500,231.2500,175.0000,14328,83.236618,1999-12-01,1999,NHTC,-0.548387,-0.991071,-0.937143,NaN,NaN
5,5,3.125,1.5625,1.5625,1.5625,21806,0.743184,2000-12-01,2000,NHTC,-0.991071,0.920000,5.176000,-0.999927,NaN
6,6,4.000,2.0000,4.0000,3.0000,256538,1.426914,2001-12-01,2001,NHTC,0.920000,-0.333333,-0.500000,-0.999455,NaN
7,7,3.000,1.0000,3.0000,2.0000,636654,0.951276,2002-12-01,2002,NHTC,-0.333333,4.250000,-0.395000,-0.989333,NaN
8,8,11.100,7.9100,8.1000,10.5000,968400,4.994196,2003-12-01,2003,NHTC,4.250000,0.047619,-0.971429,-0.972903,NaN
9,9,12.280,8.7000,10.3000,11.0000,729300,5.232018,2004-12-01,2004,NHTC,0.047619,-0.122727,-0.987273,-0.937143,NaN


In [17]:
from market_growth_analysis.etl import web_scraping

path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)

web_scraping.get_price_info('NHTC', ticker_file_all)

c:\Users\germa\AppData\Local\Programs\Python\Python39\lib\site-packages\market_growth_analysis\etl\web_scraping.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
c:\Users\germa\AppData\Local\Programs\Python\Python39\lib\site-packages\market_growth_analysis\etl\web_scraping.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])


,longevity,High,Low,Open,Close,Volume,Adjclose,Date,year,ticker,Growth -1,Growth +1,Growth +5,Growth -10,Growth -5
0,0,28000.000000,21500.000000,26000.000000,21500.000000,760,10226.214844,1995-12-01,1995,NHTC,NaN,-0.744186,-0.999927,NaN,NaN
1,1,12000.000000,4000.000000,11000.000000,5500.000000,329,2616.008545,1996-12-01,1996,NHTC,-0.744186,-0.965909,-0.999455,NaN,NaN
2,2,375.000000,125.000000,250.000000,187.500000,1749,89.182114,1997-12-01,1997,NHTC,-0.965909,1.066667,-0.989333,NaN,NaN
3,3,412.500000,318.750000,318.750000,387.500000,21139,184.309692,1998-12-01,1998,NHTC,1.066667,-0.548387,-0.972903,NaN,NaN
4,4,231.250000,131.250000,231.250000,175.000000,14328,83.236641,1999-12-01,1999,NHTC,-0.548387,-0.991071,-0.937143,NaN,NaN
5,5,3.125000,1.562500,1.562500,1.562500,21806,0.743184,2000-12-01,2000,NHTC,-0.991071,0.920000,5.176000,-0.999927,NaN
6,6,4.000000,2.000000,4.000000,3.000000,256538,1.426914,2001-12-01,2001,NHTC,0.920000,-0.333333,-0.500000,-0.999455,NaN
7,7,3.000000,1.000000,3.000000,2.000000,636654,0.951276,2002-12-01,2002,NHTC,-0.333333,4.250000,-0.395000,-0.989333,NaN
8,8,11.100000,7.910000,8.100000,10.500000,968400,4.994197,2003-12-01,2003,NHTC,4.250000,0.047619,-0.971429,-0.972903,NaN
9,9,12.280000,8.700000,10.300000,11.000000,729300,5.232016,2004-12-01,2004,NHTC,0.047619,-0.122727,-0.987273,-0.937143,NaN
